In [1]:
import pandas as pd

from config.development import DATABASE

In [33]:
df = pd.read_sql('articles', DATABASE, index_col = 'id')

In [3]:
df.sample(10)

,url,title,date,body,author
id,,,,,
6176,https://www.inform.kz/ru/sleduyuschim-sopernik...,Следующим соперником Виталия Кличко станет Дэв...,2012-03-05,АСТАНА. 5 марта. КАЗИНФОРМ - Чемпион мира в су...,None
5217,https://www.inform.kz/ru/26-27-aprelya-v-astan...,26-27 апреля в Астане пройдет X Евразийский ме...,2012-02-24,Юбилейный десятый Евразийский медиафорум пройд...,None
6316,https://www.inform.kz/ru/utrennyaya-sessiya-ka...,"Утренняя сессия KASE: доллар равен 147,93 тенге",2012-03-06,АЛМАТЫ. 6 марта. КАЗИНФОРМ - На Казахстанской ...,None
15524,https://www.inform.kz/ru/uchenye-nashli-klesch...,"Ученые нашли клеща, виновного в гибели миллиар...",2012-06-10,АСТАНА. 10 июня. КАЗИНФОРМ - Клещ-паразит расп...,None
681,https://www.inform.kz/ru/kandidaty-v-deputaty-...,Кандидаты в депутаты Мажилиса от Ассамблеи нар...,2012-01-11,ШЫМКЕНТ. 11 января. КАЗИНФОРМ /Татьяна Пылаева...,Татьяна Никитина
3763,https://www.inform.kz/ru/v-kazahstane-proverya...,В Казахстане проверяют готовность противотубер...,2012-02-10,АСТАНА. 10 февраля. КАЗИНФОРМ - В Министерстве...,None
32442,https://www.inform.kz/ru/amerikancy-predlozhil...,Американцы предложили новый метод восстановлен...,2012-12-02,АСТАНА. 2 декабря. КАЗИНФОРМ - Одна сессия сти...,None
11063,https://www.inform.kz/ru/neobhodimo-prorabotat...,Необходимо проработать пути транспортировки не...,2012-04-26,АТЫРАУ. 26 апреля. КАЗИНФОРМ - Подводя итоги о...,Виктор Сутягин
3764,https://www.inform.kz/ru/v-karagandinskoy-obla...,В Карагандинской области в кадровый резерв рук...,2012-02-10,КАРАГАНДА. 10 февраля. КАЗИНФОРМ - В Караганди...,None


# Text preprocessing
## Basic preprocessing
* Lower-case
* Remove numbers and non-letter characters
* Lemmatization

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import re

from tqdm.notebook import tqdm
tqdm.pandas()

from pymystem3 import Mystem
mystem = Mystem()


def preprocess(text):
    global mystem
    text = text.lower()
    text = re.sub('[\W_\d]+', ' ', text)
    text = mystem.lemmatize(text)
    text = ''.join(text)
    return text            

df['body_preprocessed'] = df.body.progress_apply(preprocess)
df['title_preprocessed'] = df.title.progress_apply(preprocess)

  0%|          | 0/32303 [00:00<?, ?it/s]

  0%|          | 0/32303 [00:00<?, ?it/s]

## Search using TF-IDF

In [35]:
df.index.values

array([    1,     2,     3, ..., 32443, 32444, 32445])

In [38]:
tfidf_body = TfidfVectorizer(
    ngram_range = (1,2),
    max_df = 0.8,
    min_df = 3,
    max_features = 10000,
)

tfidf_title = TfidfVectorizer(
    ngram_range = (1,2),
    max_df = 0.8,
    min_df = 3,
    max_features = 5000,
)

tfidf_index = df.index.values
tfidf_body_matrix = tfidf_body.fit_transform(df.body_preprocessed)
tfidf_title_matrix = tfidf_title.fit_transform(df.title_preprocessed)

### Get top __n__ words the article

In [7]:
n, top_n = 1, 5
a_ind = df.sample(n).index
articles = df.body_preprocessed[a_ind]
def top_n_tokens(tfidf_body: TfidfVectorizer, articles: pd.Series, top_n: int) -> list:
    a_tfidf = tfidf_body.transform(articles).todense().A
    a_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :n]
    m = tfidf_body.get_feature_names()
    
    a_tfidf = tfidf_body.transform(articles).todense().A
    a_ind_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :top_n]
    fn = tfidf_body.get_feature_names()

    return [(indT, [(fn[i], a_tfidf[ind, i]) for i in a_ind_srt[ind]]) for ind, indT in enumerate(articles.index)]

top_n_tokens(tfidf_body, articles, top_n)

[(2294,
  [('сборная', 0.39223057079815954),
   ('матч', 0.3612266905536463),
   ('со счет', 0.2364300432213877),
   ('чемпионат азия', 0.18873499864521773),
   ('победа', 0.177628618367849)])]

In [39]:
from sklearn.metrics.pairwise import linear_kernel

q = "Если хочешь выиграть меня, то сука"

def get_top_n(query, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, tfidf_index, n=10):
    q_transformed = preprocess(q)
    q_transformed_body = tfidf_body.transform([q_transformed])
    q_transformed_title = tfidf_title.transform([q_transformed])

    dist_body = linear_kernel(q_transformed_body, tfidf_body_matrix).flatten()
    dist_title = linear_kernel(q_transformed_title, tfidf_title_matrix).flatten()

    b_weight = 0.3
    t_weight = 1 - b_weight

    dist_weighted = b_weight * dist_body + t_weight * dist_title
    top_n = dist_weighted.argsort()[-n:][::-1]
    return list(zip(tfidf_index[top_n], dist_weighted[top_n]))

get_top_n(q, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, tfidf_index)

[(17502, 0.35695989475378637),
 (17403, 0.3365570339254973),
 (10590, 0.2949825758394572),
 (29902, 0.2760671032121855),
 (6498, 0.25999235343192195),
 (27290, 0.25895570023446857),
 (12141, 0.2552753608379427),
 (9166, 0.23654858372284027),
 (29306, 0.23380446277504213),
 (19770, 0.23115299691742297)]

In [9]:
import pickle

In [10]:
SEARCH_FOLDER = './app/models/search/'

In [53]:
pickle.dump(tfidf_body, open(SEARCH_FOLDER + 'tfidf_body.pkl','wb'))
pickle.dump(tfidf_body_matrix, open(SEARCH_FOLDER + 'tfidf_body_matrix.pkl','wb'))
pickle.dump(tfidf_title, open(SEARCH_FOLDER + 'tfidf_title.pkl','wb'))
pickle.dump(tfidf_title_matrix, open(SEARCH_FOLDER + 'tfidf_title_matrix.pkl','wb'))
pickle.dump(tfidf_index, open(SEARCH_FOLDER + 'tfidf_index.pkl','wb'))

### Load the pickle files for search

In [12]:
del tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix

In [13]:
tfidf_body = pickle.load(open(SEARCH_FOLDER + 'tfidf_body.pkl', 'rb'))
tfidf_body_matrix = pickle.load(open(SEARCH_FOLDER + 'tfidf_body_matrix.pkl', 'rb'))
tfidf_title = pickle.load(open(SEARCH_FOLDER + 'tfidf_title.pkl', 'rb'))
tfidf_title_matrix = pickle.load(open(SEARCH_FOLDER + 'tfidf_title_matrix.pkl', 'rb'))

In [41]:
get_top_n(q, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, tfidf_index)

[(17502, 0.35695989475378637),
 (17403, 0.3365570339254973),
 (10590, 0.2949825758394572),
 (29902, 0.2760671032121855),
 (6498, 0.25999235343192195),
 (27290, 0.25895570023446857),
 (12141, 0.2552753608379427),
 (9166, 0.23654858372284027),
 (29306, 0.23380446277504213),
 (19770, 0.23115299691742297)]

In [43]:
q = 'Олимпиада в пекине'
res = get_top_n(q, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, tfidf_index)
ids, ranks = zip(*res)
print(ids)

(6205, 18605, 19599, 20066, 22327, 2247, 30348, 21790, 22225, 15279)


In [51]:
list(ids)

[6205, 18605, 19599, 20066, 22327, 2247, 30348, 21790, 22225, 15279]

In [52]:
df.loc[list(ids)]

,url,title,date,body,author
id,,,,,
6205,https://www.inform.kz/ru/v-pekine-otkrylas-ses...,В Пекине открылась сессия ВСНП,2012-03-05,ПЕКИН. 5 марта. КАЗИНФОРМ - Сегодня в Пекине в...,Руслан Сулейменов
18605,https://www.inform.kz/ru/na-olimpiade-v-london...,На Олимпиаде в Лондоне Гульнафис Айтмухамбетов...,2012-07-13,АТЫРАУ. 13 июля. КАЗИНФОРМ - Бронзовый призер ...,Виктор Сутягин
19599,https://www.inform.kz/ru/v-londone-kazahstansk...,В Лондоне казахстанская сборная планирует выст...,2012-07-25,ЛОНДОН. 25 июля. КАЗИНФОРМ - В Лондоне казахст...,None
20066,https://www.inform.kz/ru/samye-zavidnye-nevest...,Самые завидные невесты Олимпиады,2012-07-31,АСТАНА. 31 июля. КАЗИНФОРМ - Внимание зрителей...,None
22327,https://www.inform.kz/ru/v-subbotu-novym-aviam...,В субботу новым авиамаршрутом Астана-Пекин в с...,2012-08-24,"ПЕКИН. 24 августа. КАЗИНФОРМ - В субботу, 25 а...",Руслан Сулейменов
2247,https://www.inform.kz/ru/v-pekine-tschatel-no-...,В Пекине тщательно следят за качеством воздуха,2012-01-26,ПЕКИН. 26 января. КАЗИНФОРМ /Руслан Сулейменов...,Руслан Сулейменов
30348,https://www.inform.kz/ru/v-pekine-izberut-novo...,В Пекине изберут новое руководство Компартии К...,2012-11-14,ПЕКИН. 14 ноября. КАЗИНФОРМ - Сегодня в Пекине...,Руслан Сулейменов
21790,https://www.inform.kz/ru/v-pekine-otkrylsya-fo...,В Пекине открылся Форум молодежи Шанхайской ор...,2012-08-17,ПЕКИН. 17 августа. КАЗИНФОРМ - В Пекине открыл...,Мейрамбек Байгарин
22225,https://www.inform.kz/ru/ao-eyr-astana-zapuska...,АО «Эйр Астана» запускает новый авиамаршрут «А...,2012-08-23,АЛМАТЫ. 23 августа. КАЗИНФОРМ - АО «Эйр Астана...,Мустафина Сара
